In [1]:
# Loading the model
#Reading libraries
import pandas as pd
import numpy as np
#Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
#Data preparation libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler,Normalizer
!pip install factor_analyzer==0.2.3
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo, FactorAnalyzer
from statsmodels.stats.outliers_influence import variance_inflation_factor
#Modelling libraries
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
#Measure libraries
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [2]:
from keras.models import load_model

In [6]:
df2=pd.read_csv('Clean_dash.csv').drop('Unnamed: 0',axis=1)
df2.head()

,Age,History,Tremor,Leg agility,Entropy of speech timing (-),Acceleration of speech timing(-/min2),Duration of pause intervals (ms),Gapingin-between voiced\nintervals(-/min),Duration of unvoiced stops (ms),Decay of unvoiced fricatives(promile/min),Entropy of speech timing (-).1,Acceleration of speech timing (-/min2),Duration of unvoiced stops (ms).1,Rate of speech respiration (- /min),Latency of\nrespiratory exchange (ms).1,Status
0,72.0,0,1,0,1.564,6.05,146.0,58.65,31.38,-2.101,1.564,-2.82,22.37,13.81,127.0,0
1,72.0,0,1,0,1.564,27.52,173.0,48.26,22.38,-1.745,1.569,8.20,26.88,21.77,313.0,0
2,72.0,0,1,0,1.550,11.97,377.0,47.54,38.12,2.657,1.550,4.71,22.37,22.52,201.0,0
3,72.0,0,1,0,1.519,-2.49,360.0,13.72,44.88,-0.934,1.539,-9.09,49.37,14.37,151.0,0
4,72.0,0,1,0,1.543,6.72,211.0,42.90,47.12,-0.973,1.560,11.77,26.87,14.61,151.0,0


In [7]:
# Split and Preprocessing
X = df2.loc[:, 'Age' : 'Latency of\nrespiratory exchange (ms).1'].values
y = df2['Status'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Scaling using MinMax
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train.shape
#

(2654, 15)

In [8]:
Features=df2.columns[:-1]
Features

Index(['Age', 'History', 'Tremor', 'Leg agility',
       'Entropy of speech timing (-)', 'Acceleration of speech timing(-/min2)',
       'Duration of pause intervals (ms)',
       'Gapingin-between voiced\nintervals(-/min)',
       'Duration of unvoiced stops (ms)',
       'Decay of unvoiced fricatives(promile/min)',
       'Entropy of speech timing (-).1',
       'Acceleration of speech timing (-/min2)',
       'Duration of unvoiced stops (ms).1',
       'Rate of speech respiration (- /min)',
       'Latency of\nrespiratory exchange (ms).1'],
      dtype='object')

In [3]:
Model=load_model('our_model.h5')
Model


In [9]:
# User defined function to take patients input and give a diagnosis
def Perkinsons():
  # These are the Features
  cols=Features
  # The main DataFrame
  user_df=pd.DataFrame(columns=cols)
  user_df
  # Creating an empty break
  columns=[]
  # Iterating through evvery input to take in the value of the patients
  for i in cols:
    dp=str(input(i))
    columns.append(dp)
    print(dp)
  # User inputs
  my_df=pd.DataFrame(data=columns,index=cols)
  transposed=my_df.transpose()
  # Concating my inputs to the initial dataframe
  user_df=pd.concat([user_df,transposed],axis=0)
  user_df.dtypes

  # Taking the input for a patient at a time
  user_df

  # Changing datatypes to numeric To enable encoding of categorical features

  to_float=['Age','Entropy of speech timing (-)','Acceleration of speech timing(-/min2)',
        'Duration of pause intervals (ms)',
        'Gapingin-between voiced\nintervals(-/min)',
        'Duration of unvoiced stops (ms)',
        'Decay of unvoiced fricatives(promile/min)',
        'Entropy of speech timing (-).1',
        'Acceleration of speech timing (-/min2)',
        'Duration of unvoiced stops (ms).1',
        'Rate of speech respiration (- /min)',
        'Latency of\nrespiratory exchange (ms).1']
  user_df[to_float]=user_df[to_float].astype(float)
  user_df.dtypes
  # # Picking object types
  # # objects=user_df.select_dtypes('object')
  # # objects
  # Using numpy to encode my inputs
  user_df[['History','Tremor','Leg agility']]=np.where(user_df[['History','Tremor','Leg agility']]=='Yes',1,0)
  user_df
  # Defining my features
  X_feat=user_df.values.reshape(1, -1)
  X_feat
  # # # Predicting the status of patient 0
  scaler = MinMaxScaler()
  scaler.fit(X_train)
  # Transforming the test set
  X_feat = scaler.transform(X_feat)
  X_feat
  # # Using the Scaled features to perform prediction
#   loaded_model=pickle.load(open(filename,'rb'))
#   Diagnosis=loaded_model.predict(X_feat)
#   Diagnosis
  Diagnosis=Model.predict(X_feat)
  Diagnosis
  # Output
  if Diagnosis ==[0]:
    print(' Congratulations,You dont have Perkinsons')
  else:
    print('Sooooo Sorry: You Garit!!')
  return 

In [10]:
Perkinsons()

72
Yes
Yes
No
1.563
347
23.14
54
257
57.63
26.63
-1.756
-24.98
5.5
16.76
Sooooo Sorry: You Garit!!
